In [3]:
# data manipulation
import pandas as pd
import numpy as np

# data analysis
import xgboost
import sklearn

In [4]:
# load raw data
titanic_train = pd.read_csv('train.csv')
titanic_test  = pd.read_csv('test.csv')

titanic_train.describe()

/Users/valbertoenoc/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# selecting general features of interest

features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'SibSp', 'Parch']

data_train = titanic_train[features]
print(data_train.shape)


(891, 7)


In [11]:
# fill not available values in age with median age
def fillMissingValues(data):
    data.loc[:, 'Age'] = data.Age.fillna(data.Age.median())
    data.loc[:, 'Fare'] = data.Fare.fillna(data.Embarked.median()) 
    data.loc[:, 'Embarked'] = data.Embarked.fillna('S')
    
    return data    

# convert features to numericals
def convertFeatures(data):
    data.loc[:, 'Pclass'] = data.loc[:, 'Pclass'].astype('int')
    data.loc[:, 'Sex'] = data.Sex.map({'male': 0, 'female': 1})
    data.loc[:, 'Embarked'] = data.Embarked.map({'S':0, 'C': 1, 'Q': 2})
    
    ''' Categorize Age '''
    data.loc[(data['Age'] <= 20), 'Age'] = 0
    data.loc[(data['Age'] > 20) & (data['Age'] <= 28), 'Age'] = 1
    data.loc[(data['Age'] > 28) & (data['Age'] <= 38), 'Age'] = 2
    data.loc[(data['Age'] > 38) & (data['Age'] <= 80), 'Age'] = 3
    data.loc[(data['Age'] > 80)] = 4

    ''' Categorize Fare '''
    data.loc[(data['Fare'] <= 7.91), 'Fare'] = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.343) & (data['Fare'] <= 31), 'Fare'] = 2
    data.loc[(data['Fare'] > 31)] = 3
    
    ''' Convert data type '''
    data.loc[:, 'Age'] = data.loc[:, 'Age'].astype('int')
    data.loc[:, 'Age'] = data.loc[:, 'Fare'].astype('int')
    
    return data

fillMissingValues(data_train)
# convertFeatures(data_train)

print(data_train.describe())

           Pclass  Sex    Age   Fare  Embarked      SibSp       Parch
count  891.000000  0.0  891.0  891.0     891.0  891.00000  891.000000
mean     2.690236  NaN    0.0    0.0       0.0    0.98541    0.930415
std      0.584928  NaN    0.0    0.0       0.0    1.30462    1.310153
min      1.000000  NaN    0.0    0.0       0.0    0.00000    0.000000
25%      3.000000  NaN    0.0    0.0       0.0    0.00000    0.000000
50%      3.000000  NaN    0.0    0.0       0.0    0.00000    0.000000
75%      3.000000  NaN    0.0    0.0       0.0    3.00000    3.000000
max      3.000000  NaN    0.0    0.0       0.0    4.00000    5.000000
